In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier


In [2]:
df = pd.read_csv('../../dataset/titanic/train.csv')
df_evaluation = pd.read_csv('../../dataset/titanic/test.csv')

In [3]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [6]:
df.describe(include='all')
# PassengerId: 식별자
# Survived: 생존여부 categorical variable
# Pclass: categorical variable
# Embarked: Categorical variable
# Sibsp - Number of Siblings/Spouses Aboard. Parch - Number of Parents/Children Aboard. -> 연속형 변수
# Ticket과 Fare(운임)은 필요없는 정보라 판단하고 제거할 것
# Cabin은 결측치가 너무 많아 제거 필요

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,891.000000,891.000000,891.000000,891,891,714.000000,891.000000,891.000000,891,891.000000,204,889
unique,NaN,NaN,NaN,891,2,NaN,NaN,NaN,681,NaN,147,3
top,NaN,NaN,NaN,"de Messemaeker, Mrs. Guillaume Joseph (Emma)",male,NaN,NaN,NaN,1601,NaN,B96 B98,S
freq,NaN,NaN,NaN,1,577,NaN,NaN,NaN,7,NaN,4,644
mean,446.000000,0.383838,2.308642,NaN,NaN,29.699118,0.523008,0.381594,NaN,32.204208,NaN,NaN
std,257.353842,0.486592,0.836071,NaN,NaN,14.526497,1.102743,0.806057,NaN,49.693429,NaN,NaN
min,1.000000,0.000000,1.000000,NaN,NaN,0.420000,0.000000,0.000000,NaN,0.000000,NaN,NaN
25%,223.500000,0.000000,2.000000,NaN,NaN,20.125000,0.000000,0.000000,NaN,7.910400,NaN,NaN
50%,446.000000,0.000000,3.000000,NaN,NaN,28.000000,0.000000,0.000000,NaN,14.454200,NaN,NaN
75%,668.500000,1.000000,3.000000,NaN,NaN,38.000000,1.000000,0.000000,NaN,31.000000,NaN,NaN


In [7]:
df['Pclass'].value_counts()

3    491
1    216
2    184
Name: Pclass, dtype: int64

In [8]:
df['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [9]:
columns = list(df.columns)
# unnecessary information
columns.remove('PassengerId')
columns.remove('Name')
columns.remove('Ticket')
columns.remove('Fare')

# too many missing values
columns.remove('Cabin')

# copy columns for evaluation
columns_evaluation = copy.deepcopy(columns)
columns_evaluation.insert(0, 'PassengerId')
columns_evaluation.remove('Survived')

In [10]:
data_set = df.loc[:,columns]
evaluation_set = df_evaluation.loc[:,columns_evaluation]

new_age = data_set['Age'].fillna(data_set['Age'].mean())
data_set['Age'] = new_age

new_age_eval = evaluation_set['Age'].fillna(evaluation_set['Age'].mean())
evaluation_set['Age'] = new_age_eval

# categorial 변수들을 쪼개서 넣어줌
data_set = pd.get_dummies(data_set, columns=['Pclass', 'Sex', 'Embarked'])
evaluation_set = pd.get_dummies(evaluation_set, columns=['Pclass', 'Sex', 'Embarked'])

In [11]:
# data의 scale을 0~1로 맞춤
scaler = MinMaxScaler()

scaler.fit(data_set)
data_scaled = scaler.transform(data_set)

scaler.fit(evaluation_set.iloc[:,range(1, len(evaluation_set.columns))])
evaluation_scaled = scaler.transform(evaluation_set.iloc[:,range(1, len(evaluation_set.columns))])

train_set, test_set = train_test_split(data_scaled, test_size=0.2, random_state=42)
train_set = pd.DataFrame(train_set)
test_set = pd.DataFrame(test_set)

evaluation_scaled = pd.DataFrame(evaluation_scaled)
evaluation_scaled.insert(0, 'PassengerId', evaluation_set['PassengerId'].astype(int))

In [12]:
# Decision Tree model
dt_model = DecisionTreeClassifier()
dt_model.fit(train_set.iloc[:,1:-1], train_set.iloc[:, 0])

predict_survived = dt_model.predict(test_set.iloc[:,1:-1])
print("Accuracy Score: {}".format(accuracy_score(test_set.iloc[:, 0], predict_survived)))

Accuracy Score: 0.7597765363128491


In [13]:
# K Nearest Neighbors model
knn_model = KNeighborsClassifier()
knn_model.fit(train_set.iloc[:,1:-1], train_set.iloc[:, 0])

predict_survived = knn_model.predict(test_set.iloc[:,1:-1])
print("Accuracy Score: {}".format(accuracy_score(test_set.iloc[:, 0], predict_survived)))

Accuracy Score: 0.7988826815642458


In [14]:
# RandomForest model
rf_model= RandomForestClassifier()
rf_model.fit(train_set.iloc[:,1:-1], train_set.iloc[:, 0])

predict_survived = rf_model.predict(test_set.iloc[:,1:-1])
print("Accuracy Score: {}".format(accuracy_score(test_set.iloc[:, 0], predict_survived)))

Accuracy Score: 0.8156424581005587


In [18]:
evaluation_scaled

,PassengerId,0,1,2,3,4,5,6,7,8,9,10
0,892,0.452723,0.000,0.000000,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
1,893,0.617566,0.125,0.000000,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
2,894,0.815377,0.000,0.000000,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
3,895,0.353818,0.000,0.000000,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,896,0.287881,0.125,0.111111,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,0.396975,0.000,0.000000,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
414,1306,0.512066,0.000,0.000000,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
415,1307,0.505473,0.000,0.000000,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
416,1308,0.396975,0.000,0.000000,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0


In [23]:
predicted_survived = rf_model.predict(evaluation_scaled.iloc[:,1:-1])

result_df = pd.DataFrame()
result_df['PassengerId'] = evaluation_scaled['PassengerId']
result_df['Survived'] = pd.Series(predicted_survived).astype(int)

result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   PassengerId  418 non-null    int64
 1   Survived     418 non-null    int64
dtypes: int64(2)
memory usage: 6.7 KB


In [24]:
result_df.to_csv('../../dataset/titanic/submission.csv', index=False)